In [34]:
import datetime

import os

os.getcwd()
if '.git' not in os.listdir():
    os.chdir('./../../../../')
os.listdir()

from ml.ml_core.src.core.model_factory import model_factory

import keras_core as keras
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from tensorboard.plugins.hparams import api as hp


## Configurations

In [35]:
params_grid = {
    "model": "bidirectionnal_lstm",
    "embedder": "camembert",
    "embedding_data": "almanach-camembert-base",
    "dataset": "fr_hf.csv",
    "batch_size": 800
}

In [36]:
data_folder_src = 'ml/ml_core/data/processed/'
ml_core_folder_path = 'ml/ml_core/'
data_path_src = f'{data_folder_src}{params_grid["dataset"]}'
date_str = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
experiment_name = f'deepwoke_{params_grid["embedder"]}_{params_grid["model"]}_{params_grid["dataset"]}_{date_str}'

log_dir = f"{ml_core_folder_path}log/fit/{experiment_name}"
model_weight_dst = f'{ml_core_folder_path}model_weights/{experiment_name}-model.keras'

In [37]:

vector_path = f'{ml_core_folder_path}embedded_vector/{params_grid["embedding_data"]}_{params_grid["dataset"]}'
vector_x_dst = f'{vector_path}.x.npy'
vector_y_dst = f'{vector_path}.y.npy'

# loading dataset

In [38]:
X = np.load(vector_x_dst)
y = np.load(vector_y_dst)
X = np.expand_dims(X, axis=1)

# Model

## callbacks

In [39]:
early_stopping = keras.callbacks.EarlyStopping(
    patience=10,
    min_delta=0.001,
    restore_best_weights=True,
    monitor='loss'
)


In [40]:

class EarlyStoppingLogging(keras.callbacks.Callback):
    def __init__(self, early_stopping_callback, log_dir):
        super().__init__()
        self.early_stopping = early_stopping_callback
        self.stopped_epoch = 0
        self.writer = tf.summary.create_file_writer(log_dir)

    def on_epoch_end(self, epoch, logs=None):
        if self.early_stopping.stopped_epoch > 0:
            self.stopped_epoch = self.early_stopping.stopped_epoch
            with self.writer.as_default():
                tf.summary.scalar('early_stopping_epoch', self.stopped_epoch, step=epoch)
                self.writer.flush()


In [41]:
tensorboard_callback = keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
early_stopping_logging_callback = EarlyStoppingLogging(early_stopping, log_dir)

In [42]:
callback = [
    early_stopping,
    tensorboard_callback,
    hp.KerasCallback(log_dir, params_grid),
    early_stopping_logging_callback,
]

# Model building

In [43]:
def model_builder(input_dim, output_dim):
    activation_function = 'softmax'
    loss = 'categorical_crossentropy'
    if output_dim == 1:
        activation_function = 'sigmoid'
        loss = 'binary_crossentropy'
    print(input_dim, output_dim)


    input_tensor = keras.layers.Input(shape=input_dim)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True, activation='relu'))(input_tensor)
    hl = keras.layers.Dropout(0.2)(hl)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(1024, return_sequences=True, activation='relu'))(hl)
    hl = keras.layers.Dropout(0.2)(hl)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(1025, return_sequences=True, activation='relu'))(hl)
    hl = keras.layers.Dropout(0.2)(hl)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True, activation='relu'))(hl)
    hl = keras.layers.Dropout(0.2)(hl)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(512, return_sequences=True, activation='relu'))(hl)
    hl = keras.layers.Dropout(0.2)(hl)

    hl = keras.layers.Bidirectional(keras.layers.LSTM(512, activation='relu'))(hl)
    hl = keras.layers.Dropout(0.2)(hl)
    
    
    hl = keras.layers.Dense(512, activation='relu')(hl)
    hl = keras.layers.Dense(256, activation='relu')(hl)
    output_tensor = keras.layers.Dense(output_dim, activation=activation_function)(hl)

    model_ = keras.models.Model(inputs=[input_tensor], outputs=[output_tensor])
    model_.compile(optimizer='adam', loss=loss, metrics=['accuracy'])

    return model_

### training the model

In [44]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1, random_state=42)
input_dim = X_train.shape[1:]
model = model_builder(input_dim, 1)
print(model.summary)

tf.compat.v1.global_variables_initializer()

(1, 768) 1
<bound method Model.summary of <Functional name=functional_9, built=True>>


In [45]:
model.fit(X_train,
          y_train,
          epochs=500,
          batch_size=params_grid['batch_size'],
          callbacks=callback,
          validation_data=(X_test, y_test)
          )

Epoch 1/500
19/21 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step - accuracy: 0.6686 - loss: 0.6560

2024-07-07 19:47:44.344906: I tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:328] ptxas warning : Registers are spilled to local memory in function 'fusion_432', 16 bytes spill stores, 16 bytes spill loads



21/21 ━━━━━━━━━━━━━━━━━━━━ 0s 589ms/step - accuracy: 0.6750 - loss: 0.6516

2024-07-07 19:47:49.322922: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 754974720 exceeds 10% of free system memory.
2024-07-07 19:47:49.924530: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1006632960 exceeds 10% of free system memory.
2024-07-07 19:47:50.369630: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 754974720 exceeds 10% of free system memory.
2024-07-07 19:47:50.599324: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 1006632960 exceeds 10% of free system memory.
2024-07-07 19:47:51.062650: W tensorflow/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 2013265920 exceeds 10% of free system memory.


21/21 ━━━━━━━━━━━━━━━━━━━━ 45s 1s/step - accuracy: 0.6778 - loss: 0.6496 - val_accuracy: 0.7622 - val_loss: 0.5486
Epoch 2/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 383ms/step - accuracy: 0.7588 - loss: 0.5548 - val_accuracy: 0.7622 - val_loss: 0.5488
Epoch 3/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 352ms/step - accuracy: 0.7666 - loss: 0.5432 - val_accuracy: 0.7622 - val_loss: 0.5487
Epoch 4/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 348ms/step - accuracy: 0.7575 - loss: 0.5541 - val_accuracy: 0.7622 - val_loss: 0.5487
Epoch 5/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 381ms/step - accuracy: 0.7589 - loss: 0.5520 - val_accuracy: 0.7622 - val_loss: 0.5264
Epoch 6/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 350ms/step - accuracy: 0.7656 - loss: 0.5100 - val_accuracy: 0.7939 - val_loss: 0.4585
Epoch 7/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 8s 363ms/step - accuracy: 0.7871 - loss: 0.4596 - val_accuracy: 0.8054 - val_loss: 0.4306
Epoch 8/500
21/21 ━━━━━━━━━━━━━━━━━━━━ 7s 341ms/step - accuracy: 0.8001 - loss: 0.4385 - val_accuracy: 0.8076 -

# save model

In [46]:
model.save(model_weight_dst)